# EDA IMAGENES

In [53]:
import numpy as np
import pandas as pd
import os
from PIL import Image
import shutil

In [54]:
def get_jpg_paths(directory):
    """
    Recorre una carpeta y sus subcarpetas para encontrar archivos con extensión .jpeg.
    Retorna un DataFrame con una columna 'relative_path' que contiene los caminos relativos de las imágenes.
    
    Parameters:
        directory (str): El path a la carpeta raíz que contiene las subcarpetas.
    
    Returns:
        pd.DataFrame: Un DataFrame con los caminos relativos de las imágenes .jpeg.
    """
    jpg_paths = []
    
    # Recorre todas las carpetas y archivos dentro del directorio
    for root, _, files in os.walk(directory):
        for file in files:
            if file.lower().endswith('.jpg'):  # Aseguramos que la extensión sea .jpeg
                # Calculamos el camino relativo
                relative_path = os.path.relpath(os.path.join(root, file), start=directory)
                jpg_paths.append(relative_path)
    
    # Creamos un DataFrame con los resultados
    df = pd.DataFrame(jpg_paths, columns=['relative_path'])
    return df

In [55]:
directory = r"C:\Users\Adrian\Documents\MONDRAGON\PBL2 Grupo 02\Mantenimiento_Predictivo_PBL_grupo02\product_fault"  # Ruta de la carpeta raíz
df = get_jpg_paths(directory)

In [56]:
df

,relative_path
0,kos01\Part0.jpg
1,kos01\Part1.jpg
2,kos01\Part2.jpg
3,kos01\Part3.jpg
4,kos01\Part4.jpg
...,...
394,kos50\Part3.jpg
395,kos50\Part4.jpg
396,kos50\Part5.jpg
397,kos50\Part6.jpg


Tenemos 399 productos a los que se les ha puesto una máscara para saber si tienen defecto o no vamos a generar un dataframe con 

In [57]:
import os

import pandas as pd

def get_jpg_paths_with_defects(directory):
    """
    Busca archivos .jpeg en una carpeta y sus subcarpetas, verifica si su archivo .bmp correspondiente tiene píxeles blancos.
    El archivo .bmp correspondiente tiene el mismo nombre que el .jpeg pero con el sufijo '_label'.
    Devuelve un DataFrame con las columnas 'relative_path' y 'defecto'.
    
    Parameters:
        directory (str): El path a la carpeta raíz que contiene las subcarpetas.
    
    Returns:
        pd.DataFrame: Un DataFrame con los caminos relativos de las imágenes .jpeg y la columna 'defecto'.
    """
    jpg_paths = []
    defects = []

    # Recorre todas las carpetas y archivos dentro del directorio
    for root, _, files in os.walk(directory):
        for file in files:
            if file.lower().endswith('.jpg'):  # Aseguramos que la extensión sea .jpeg
                # Calculamos el camino relativo del archivo .jpeg
                relative_path_jpg = os.path.relpath(os.path.join(root, file), start=directory)
                jpg_paths.append(relative_path_jpg)

                # Verificamos si el archivo .bmp correspondiente existe
                bmp_filename = os.path.splitext(file)[0] + "_label.bmp"  # Añadimos '_label' al nombre base
                bmp_filepath = os.path.join(root, bmp_filename)
                
                if os.path.exists(bmp_filepath):
                    # Abrimos la imagen .bmp y verificamos si contiene píxeles blancos
                    try:
                        with Image.open(bmp_filepath) as bmp_image:
                            # Convertimos la imagen a modo 'RGB' si no lo está
                            if bmp_image.mode != 'RGB':
                                bmp_image = bmp_image.convert('RGB')
                            # Extraemos los datos de los píxeles
                            pixels = list(bmp_image.getdata())
                            # Buscamos un píxel blanco (255, 255, 255)
                            has_white_pixel = any(pixel == (255, 255, 255) for pixel in pixels)
                            defects.append(1 if has_white_pixel else 0)
                    except Exception as e:
                        print(f"Error procesando el archivo {bmp_filepath}: {e}")
                        defects.append(1)  # Consideramos que no tiene defecto en caso de error
                else:
                    defects.append(1)  # Si no existe el archivo .bmp, asumimos sin defecto

    # Creamos un DataFrame con los resultados
    df = pd.DataFrame({
        'relative_path': jpg_paths,
        'defecto': defects
    })
    return df

In [58]:
directory = r"C:\Users\Adrian\Documents\MONDRAGON\PBL2 Grupo 02\Mantenimiento_Predictivo_PBL_grupo02\product_fault"  # Ruta de la carpeta raíz
df = get_jpg_paths_with_defects(directory)
print(df)

       relative_path  defecto
0    kos01\Part0.jpg        0
1    kos01\Part1.jpg        0
2    kos01\Part2.jpg        0
3    kos01\Part3.jpg        0
4    kos01\Part4.jpg        0
..               ...      ...
394  kos50\Part3.jpg        0
395  kos50\Part4.jpg        1
396  kos50\Part5.jpg        0
397  kos50\Part6.jpg        0
398  kos50\Part7.jpg        0

[399 rows x 2 columns]


In [59]:
df['defecto'].value_counts()

defecto
0    347
1     52
Name: count, dtype: int64

Como vemos, el dataset esta muy desbalanceado ya que tenemos 52 piezas con defecto y 347 sin defecto. Posteriormente se valorará si hacer oversampling de las piezas con defecto.

In [60]:
# Guardamos el DataFrame en un archivo CSV
df.to_csv("defectos_info.csv", index=False)

In [61]:
def augment_defective_images(df, source_dir, target_dir="data_augmentation"):
    """
    Copia todas las imágenes .jpg a 'data_augmentation'.
    - Si defecto == 1, genera:
      - Imagen rotada 180° (*_rot.jpg)
      - Imagen espejo horizontal (*_res.jpeg)
    - Si defecto == 0, copia el archivo *_label.bmp si existe.

    Parameters:
        df (pd.DataFrame): DataFrame con las columnas 'relative_path' y 'defecto'.
        source_dir (str): Directorio raíz donde están las imágenes originales.
        target_dir (str): Directorio donde se guardarán las imágenes aumentadas.
    """
    for index, row in df.iterrows():
        source_path = os.path.join(source_dir, row['relative_path'])
        
        # Obtener la ruta relativa y el nombre del archivo sin extensión
        relative_dir = os.path.dirname(row['relative_path'])
        filename_wo_ext = os.path.splitext(os.path.basename(row['relative_path']))[0]
        
        # Ruta para copiar la imagen original en data_augmentation
        original_aug_path = os.path.join(target_dir, relative_dir, f"{filename_wo_ext}.jpg")
        
        # Crear la estructura de carpetas en data_augmentation
        os.makedirs(os.path.dirname(original_aug_path), exist_ok=True)
        
        try:
            with Image.open(source_path) as img:
                # Copiar la imagen original
                img.save(original_aug_path)
                print(f"Imagen original guardada en: {original_aug_path}")
                
                if row['defecto'] == 1:
                    # Si tiene defecto, generar y guardar imagen rotada 180°
                    rot_aug_path = os.path.join(target_dir, relative_dir, f"{filename_wo_ext}_rot.jpg")
                    rotated_img = img.rotate(180)
                    rotated_img.save(rot_aug_path)
                    print(f"Imagen rotada guardada en: {rot_aug_path}")
                    
                    # Generar y guardar imagen espejo horizontal
                    res_aug_path = os.path.join(target_dir, relative_dir, f"{filename_wo_ext}_res.jpg")
                    flipped_img = img.transpose(Image.FLIP_LEFT_RIGHT)
                    flipped_img.save(res_aug_path)
                    print(f"Imagen espejo guardada en: {res_aug_path}")
                
                elif row['defecto'] == 0:
                    # Si no tiene defecto, copiar el archivo *_label.bmp si existe
                    bmp_filename = f"{filename_wo_ext}_label.bmp"
                    bmp_source_path = os.path.join(source_dir, relative_dir, bmp_filename)
                    bmp_aug_path = os.path.join(target_dir, relative_dir, bmp_filename)
                    
                    if os.path.exists(bmp_source_path):
                        shutil.copy2(bmp_source_path, bmp_aug_path)
                        print(f"Archivo .bmp copiado en: {bmp_aug_path}")
                    else:
                        print(f"Archivo .bmp no encontrado para: {bmp_source_path}")
                    
        except Exception as e:
            print(f"Error al procesar {source_path}: {e}")

In [62]:
augment_defective_images(df, directory, target_dir="data_augmentation")

Imagen original guardada en: data_augmentation\kos01\Part0.jpg
Archivo .bmp copiado en: data_augmentation\kos01\Part0_label.bmp
Imagen original guardada en: data_augmentation\kos01\Part1.jpg
Archivo .bmp copiado en: data_augmentation\kos01\Part1_label.bmp
Imagen original guardada en: data_augmentation\kos01\Part2.jpg
Archivo .bmp copiado en: data_augmentation\kos01\Part2_label.bmp
Imagen original guardada en: data_augmentation\kos01\Part3.jpg
Archivo .bmp copiado en: data_augmentation\kos01\Part3_label.bmp
Imagen original guardada en: data_augmentation\kos01\Part4.jpg
Archivo .bmp copiado en: data_augmentation\kos01\Part4_label.bmp
Imagen original guardada en: data_augmentation\kos01\Part5.jpg
Imagen rotada guardada en: data_augmentation\kos01\Part5_rot.jpg
Imagen espejo guardada en: data_augmentation\kos01\Part5_res.jpg
Imagen original guardada en: data_augmentation\kos01\Part6.jpg
Archivo .bmp copiado en: data_augmentation\kos01\Part6_label.bmp
Imagen original guardada en: data_augme

In [63]:
df=get_jpg_paths_with_defects('data_augmentation')
print(df)

           relative_path  defecto
0        kos01\Part0.jpg        0
1        kos01\Part1.jpg        0
2        kos01\Part2.jpg        0
3        kos01\Part3.jpg        0
4        kos01\Part4.jpg        0
..                   ...      ...
498  kos50\Part4_res.jpg        1
499  kos50\Part4_rot.jpg        1
500      kos50\Part5.jpg        0
501      kos50\Part6.jpg        0
502      kos50\Part7.jpg        0

[503 rows x 2 columns]


In [64]:
# Guardamos el DataFrame en un archivo CSV
df.to_csv("defectos_info.csv", index=False)

In [65]:
df.value_counts('defecto')

defecto
0    347
1    156
Name: count, dtype: int64

Con este data augmentation hemos generado más errorores a partir de imagen giradas 180º y espejo; vacilitando a la hora del entranimiento tener un conjunto de datos más equilibrado.